Lets see what the data contains and how to handle it.

In [3]:
with open('../data/input.txt', 'r', encoding='utf-8') as f:
    data = f.read()

In [9]:
print("Data Type: ", type(data))
print("Number of characters in data: ", len(data))
print("\nFirst 500 characters of the data:\n ===============================\n", data[:400])

Data Type:  <class 'str'>
Number of characters in data:  1115394

First 500 characters of the data:
 First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it 


Andrej Karpathy suggest firt trying to make a generative model based on character level tokenization as that will result in a smaller number of word and super simple tokenization.

In [59]:
import numpy as np

# Create a list of unique characters
chars = sorted(list(set(data)))
vocab_size = len(chars)

# Create dictionaries for character-to-index and index-to-character conversions
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# Print the total number of unique characters (also known as the vocabulary size)
print('Number of unique characters (Vocabulary size): ', vocab_size)

# Print some of the items from our dictionaries
print("\nSome items in our character-to-index dictionary:\n", {k: char_to_ix[k] for k in list(char_to_ix)[:10]})
print("\nSome items in our index-to-character dictionary:\n", {k: ix_to_char[k] for k in list(ix_to_char)[:10]})

# Print the first 500 indices
print("\nFirst 500 indices:\n", data_ix[:400])

# Encoder and Decode
encode = lambda s: [char_to_ix[c] for c in s]
decode = lambda l: ''.join(ix_to_char[i] for i in l)


Number of unique characters (Vocabulary size):  65

Some items in our character-to-index dictionary:
 {'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9}

Some items in our index-to-character dictionary:
 {0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3'}

First 500 indices:
 [18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1 39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39
 58 46 43 56  1 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47
 57 46 12  0  0 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53
 50 60 43 42  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47
 56 57 58  6  1 63 53 59  1 49 52 53 61  1 15 39 47 59 57  1 25 39 56 41
 47 5

Now that we have simple encoder and decoder function for tokenizartion of the text. Lets tokenize the text into a torch tensor

In [126]:
import torch
torch.manual_seed(1337)
td = torch.tensor(encode(data))

# Determine the split index for 80-20 train-validation split
split_idx = int(0.8 * len(td))

# Split the data into training and validation sets
train_data = td[:split_idx]
val_data = td[split_idx:]

print("Size of training set: ", len(train_data))
print("Size of validation set: ", len(val_data))

Size of training set:  892315
Size of validation set:  223079


Next lets look into the consept of the context when predicting the next token. As we are working with a purely generative model here our next token generation will sole depent on the prevous tokens.

In [127]:
block_size = 8

In [128]:
#predition y is the token at index+1
x = train_data
y = train_data[1:len(train_data)+1]
print("\nExample of context for each token prediction:")
for i in range(block_size):
    context = (x[0:i+1])
    target = (y[i])
    print(f"Context: {context} -> Target: {target}")




Example of context for each token prediction:
Context: tensor([18]) -> Target: 47
Context: tensor([18, 47]) -> Target: 56
Context: tensor([18, 47, 56]) -> Target: 57
Context: tensor([18, 47, 56, 57]) -> Target: 58
Context: tensor([18, 47, 56, 57, 58]) -> Target: 1
Context: tensor([18, 47, 56, 57, 58,  1]) -> Target: 15
Context: tensor([18, 47, 56, 57, 58,  1, 15]) -> Target: 47
Context: tensor([18, 47, 56, 57, 58,  1, 15, 47]) -> Target: 58


For effective training on multi thredded device(GPU) it is best to introduce batches. One batch contains multiple blocks of context and predition targets. Thus the predition can be formulated as a matrix operation that can be parallelized efectively.

In [129]:

from typing import Tuple
block_size = 8
batch_size = 4 
def create_batches(
        x: torch.Tensor, 
        y: torch.Tensor, 
        batch_size: int, 
        block_size: int
    ) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    This function creates batches of data for training. It randomly selects indices for the batches, 
    then stacks the corresponding elements from the input data (x) and target data (y) into tensors.
    
    Parameters:
    x (torch.Tensor): The input data.
    y (torch.Tensor): The target data.
    batch_size (int): The number of samples per batch.
    block_size (int): The number of elements per sample.
    
    Returns:
    x (torch.Tensor): The batched input data.
    y (torch.Tensor): The batched target data.
    """
    
    idx = torch.randint(len(x)-block_size, (batch_size,))
    print(idx)

    x = torch.stack([x[i:i+block_size] for i in idx])
    y = torch.stack([y[i:i+block_size] for i in idx])
    return x, y

xt, yt = create_batches(x, y, batch_size, block_size)
print("Information about xt:")
print("Type: ", type(xt))
print("Shape: ", xt.shape)
print("Elements: ", xt)

print("\nInformation about yt:")
print("Type: ", type(yt))
print("Shape: ", yt.shape)
print("Elements: ", yt)


tensor([188288,  12671, 712251, 226939])
Information about xt:
Type:  <class 'torch.Tensor'>
Shape:  torch.Size([4, 8])
Elements:  tensor([[58, 63,  8,  0,  0, 19, 24, 27],
        [39, 59, 45, 46, 58,  1, 46, 43],
        [49, 43, 57,  1, 53, 50, 42,  1],
        [52, 41, 47, 43, 52, 58,  1, 56]])

Information about yt:
Type:  <class 'torch.Tensor'>
Shape:  torch.Size([4, 8])
Elements:  tensor([[63,  8,  0,  0, 19, 24, 27, 33],
        [59, 45, 46, 58,  1, 46, 43,  1],
        [43, 57,  1, 53, 50, 42,  1, 46],
        [41, 47, 43, 52, 58,  1, 56, 47]])


Lets try to understand the training and predition methods using a simple bigram model.

In [236]:
from torch import nn

class BigramModel(nn.Module):

    def __init__(self, vocab_size: int) -> None:
        super().__init__()
        self.token_embed_table = nn.Embedding(vocab_size,vocab_size)
    
    def forward(self, id: int , targets: torch.Tensor=None):
        logits = self.token_embed_table(id)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) #reshape tensor to combine batch and block(timestep) dimension
            targets =targets.view(B*T)
            loss = nn.functional.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, id: int, num_of_tokens=100):
        for _ in range(num_of_tokens):
            logits, loss = self(id) #Addidg the whole context is useless here as the model is bigram,
            logits = logits[:,-1,:]
            logits = nn.functional.softmax(logits, dim=-1)
            result = torch.multinomial(logits, num_samples=1)
            id = torch.cat((id,result), dim=1)
        return id

m = BigramModel(vocab_size)
logits, loss = m(xt, yt)
print("The shape of the prediction (batch* block(time), n_channels): ", logits.shape)
print("The score of the first token of the first batch for context of one(first time step):\n", logits[0])
print("Loss: ", loss)
new_token = m.generate(xt)
print("Generating new tokens")
print("The shape of the generations: ", new_token.shape)
print("The generation looks as following:\n", new_token)
print("The one of the generations decoded to characters insted of tokens: \n", decode(new_token[0].tolist()))

The shape of the prediction (batch* block(time), n_channels):  torch.Size([32, 65])
The score of the first token of the first batch for context of one(first time step):
 tensor([-1.2757,  1.3091,  1.2854, -0.1952,  1.1449,  1.4121,  0.1026,  0.3573,
         0.1778,  2.2240, -0.4292, -0.2717, -1.5658, -1.5595,  0.6576,  0.3901,
        -2.2354,  0.3967, -0.8899, -1.4107,  0.3521, -0.3940,  2.1606,  0.5367,
        -0.0440, -0.1164,  0.5739,  0.4141,  1.2841,  0.9807,  0.2497,  0.8167,
        -0.3198, -0.2923, -1.2555,  0.4961, -0.5637, -0.1596,  0.2315,  1.6227,
         0.4444,  0.1983,  1.3251, -1.2757,  0.7441,  1.0138,  1.4430, -1.1215,
        -0.0633, -1.1977, -0.8633,  0.5283,  1.2441,  1.7976,  0.4072, -0.4093,
         0.0119,  0.2657, -1.6750,  0.6776,  1.3463,  0.9899, -1.4355,  0.9548,
         0.1165], grad_fn=<SelectBackward0>)
Loss:  tensor(4.1808, grad_fn=<NllLossBackward0>)
Generating new tokens
The shape of the generations:  torch.Size([4, 108])
The generation looks 

## Training the the model

currently the model is a simple bigram model but still it should be able to learn a little text like structure

In [253]:
import torch.optim as optim

# Define the optimizer
optimizer = optim.Adam(m.parameters(), lr=0.001)

# Define the number of epochs
epochs = 10000

# Training loop
for epoch in range(epochs):
    # Zero the gradients
    optimizer.zero_grad()
    # Forward pass
    logits, loss = m(xt, yt)
    # Backward pass
    loss.backward()
    # Update the weights
    optimizer.step()
    # Print the loss for this epoch
    print(f"Epoch {epoch+1}/{epochs} Loss: {loss.item()}")


Epoch 1/10000 Loss: 0.5683945417404175
Epoch 2/10000 Loss: 0.5659019947052002
Epoch 3/10000 Loss: 0.5634489059448242
Epoch 4/10000 Loss: 0.5610343813896179
Epoch 5/10000 Loss: 0.5586596131324768
Epoch 6/10000 Loss: 0.5563235878944397
Epoch 7/10000 Loss: 0.5540269017219543
Epoch 8/10000 Loss: 0.5517696738243103
Epoch 9/10000 Loss: 0.5495518445968628
Epoch 10/10000 Loss: 0.547373354434967
Epoch 11/10000 Loss: 0.5452338457107544
Epoch 12/10000 Loss: 0.5431332588195801
Epoch 13/10000 Loss: 0.5410711765289307
Epoch 14/10000 Loss: 0.5390474200248718
Epoch 15/10000 Loss: 0.5370621085166931
Epoch 16/10000 Loss: 0.5351144671440125
Epoch 17/10000 Loss: 0.5332044959068298
Epoch 18/10000 Loss: 0.5313318967819214
Epoch 19/10000 Loss: 0.5294961929321289
Epoch 20/10000 Loss: 0.5276970267295837
Epoch 21/10000 Loss: 0.5259340405464172
Epoch 22/10000 Loss: 0.524206817150116
Epoch 23/10000 Loss: 0.5225149989128113
Epoch 24/10000 Loss: 0.5208578705787659
Epoch 25/10000 Loss: 0.5192351341247559
Epoch 26/10

In [255]:
new_token = m.generate(xt)
print("Generating new tokens")
print("The sample of generation decoded to characters:\n", decode(new_token[0].tolist()))

Generating new tokens
The sample of generation decoded to characters:
 ty.

GLOUXT,nt ries ht ht he hty.
GLOUW?nciencies ht riencies hty.



GLOUmnt rie hes hes ries old ries henc


The out put is somewhat less random than what we got on untrained model. So the training seems to be working as the validation seems to suggest.

## Attention related math
Next up lets look into the details of how the attendtion works. We will be looking only at generating attention so next we will look at how the information is passed from the past tokens to the current prediction.


In [290]:
torch.manual_seed(1010)
_B, _T, _C = 3, 6, 2
x = torch.rand(_B,_T,_C)
x.shape

torch.Size([3, 6, 2])

In [268]:
#The previous characters will affect the prediction for each token in the block_size(time) T. The calculation has to be done for each block in the batch. So there are too loops:
xbow = torch.zeros((_B,_T,_C))
for b in range(_B):
    for t in range(_T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [267]:
print("X values:\n", x[0])
print("The averrage of the past X values: \n",xbow[0])

X values:
 tensor([[0.9297, 0.8741],
        [0.9517, 0.6191],
        [0.6127, 0.0175],
        [0.1215, 0.6891],
        [0.3499, 0.3823],
        [0.0396, 0.8546]])
The averrage of the past X values: 
 tensor([[0.9297, 0.8741],
        [0.9407, 0.7466],
        [0.8313, 0.5036],
        [0.6539, 0.5499],
        [0.5931, 0.5164],
        [0.5008, 0.5728]])


As the loop are independent of each other, these can be made into a matrix operation as follows

In [281]:
#upper triangural matrix can be used to cancel out the future tokens in the block for inter fering with the current prediction
#NOTE: Not using the tril can be usefull in the encoder part of a machine translation where the future tokens of the input have an effect on the current prediction
# see the Attention is all you need paper for the original idea on that.
a = torch.tril(torch.ones(3,3))
b = torch.randint(0,10,(3,6)).float()
print ("a:\n",a)
print ("b:\n",b)
print ("a dot b", a @ b)

a:
 tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
b:
 tensor([[9., 3., 6., 9., 3., 6.],
        [3., 6., 5., 2., 5., 3.],
        [2., 8., 4., 8., 6., 0.]])
a dot b tensor([[ 9.,  3.,  6.,  9.,  3.,  6.],
        [12.,  9., 11., 11.,  8.,  9.],
        [14., 17., 15., 19., 14.,  9.]])


As can be seen this lead us with matrix that looks quite good, except that we are counting sums not averages. If we normalize each row of the tril to sum up to 1 that should do the trick

In [287]:
a = a / torch.sum(a, 1, keepdim=True)
print ("a:\n",a)
print ("b:\n",b)
print ("a dot b", a @ b)

a:
 tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b:
 tensor([[9., 3., 6., 9., 3., 6.],
        [3., 6., 5., 2., 5., 3.],
        [2., 8., 4., 8., 6., 0.]])
a dot b tensor([[9.0000, 3.0000, 6.0000, 9.0000, 3.0000, 6.0000],
        [6.0000, 4.5000, 5.5000, 5.5000, 4.0000, 4.5000],
        [4.6667, 5.6667, 5.0000, 6.3333, 4.6667, 3.0000]])


So this in the original loop looks like this

In [296]:
weights = torch.tril(torch.ones(_T,_T))
weights = weights / weights.sum(1, keepdim=True)
xbox2 = weights @ x  #(T,T) @ (B, T, C) --auto baching--> (B,T,T) @ (B,T,C) = (B,T,C)
torch.allclose(xbow, xbox2) # matching the loop results

True

This can stil be improved a bit by using softmax

In [304]:
tril = torch.tril(torch.ones(_T,_T))
weights = torch.ones(_T, _T)
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = nn.functional.softmax(weights)
xbow3 = weights @ x
torch.allclose(xbow, xbow3)

/var/folders/qc/syfsz0rd6bl0jt8_xl2vr_s80000gn/T/ipykernel_27695/401844102.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = nn.functional.softmax(weights)


True

## Self attention